In [4]:
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
import re
import pandas as pd
import swifter
from bertopic import BERTopic
stop_words = list(set(stopwords.words('english')))

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[{}0-9]'.format(string.punctuation), ' ', text)
    text=re.sub(r'[^A-Za-z0-9 ]+', ' ', text)
    text = word_tokenize(text)
    text = [word for word in text if word not in stop_words]
    text = [WordNetLemmatizer().lemmatize(word) for word in text]
    text = ' '.join(text)
    return text

In [5]:
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import swifter

newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))
df=pd.DataFrame({"content":newsgroups["data"]})

#df=df.sample(1000) #for a sample of 1000 documents
df["content"]=df["content"].swifter.apply(lambda x: preprocess_text(x))
df['content_length'] = df['content'].str.len()

df = df[df['content_length'] > 100]
df = df[df['content_length'] < 2000]

df=df[["content"]].reset_index(drop=True).reset_index().rename(columns={"index":"id"})
documents=df.content.to_list()

Pandas Apply:   0%|          | 0/18846 [00:00<?, ?it/s]

In [3]:



topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents)
topic_model.save("bertopic_20newsgroup")

In [5]:
def remove_punct(text):
    text  = "".join([char for char in text if char not in string.punctuation])
    text = re.sub('[0-9]+', '', text)
    return text

def count_remover(text,threshold=4):
    if len(text.split())<threshold:
        return pd.NaT
    else:
        return text

data=pd.read_json('tweets.json' ,lines=True)
df=data[["Text","CreatedAt"]].rename(columns={"Text":"content","CreatedAt":"time"})
df['content'] = df['content'].str.replace(r'@\w+', '')
df['content'] = df['content'].apply(lambda x: re.sub(r"http\S+", "", x))
df['content'] = df['content'].apply(lambda x: remove_punct(x))
df['content'] = df['content'].apply(lambda x: count_remover(x))
df=df.dropna()
df["content"]=df["content"].swifter.apply(lambda x: preprocess_text(x))
df=df.dropna()
topic_model = BERTopic()
topics, probs = topic_model.fit_transform(df.content.tolist())
#topic_model.save("bertopic_EM_tweets")

Pandas Apply:   0%|          | 0/14268 [00:00<?, ?it/s]

In [ ]:
topic_model.get_topics()

In [3]:
from bertopic import BERTopic
topic_model = BERTopic.load("results/bertopic_20newsgroup")

In [6]:
topics=[]
t=topic_model.get_topics()
for i in range(len(t)-1):
    topic=[]
    for w in t[i]:
        topic.append(w[0])
    topics.append(topic)
print(len(topics))

217


In [7]:
import pickle
with open("results/bertopic_EM_tweets_topics", "wb") as fp:   #Pickling
     pickle.dump(topics, fp)

In [17]:
document_topic_distribution=topic_model.transform(documents)[0]

In [18]:
import numpy as np
documents_distribution=[]
for i,document in enumerate(document_topic_distribution):
    document_dist=np.zeros(len(topics))
    if document>-1:
        document_dist[document]=1
    documents_distribution.append(document_dist)
documents_topic_distribution_array = np.vstack(documents_distribution)

In [19]:

with open("results/bertopic_EM_tweets_top_doc_dist", "wb") as fp:   #Pickling
     pickle.dump(documents_topic_distribution_array, fp)